In [45]:
from math import log
import treePlotter as tpl
import operator
import csv
import numpy as np
from sklearn import datasets
import seaborn

def createDataSet():
    dataSett = seaborn.load_dataset("iris")
    dataSettt = list(dataSett)
    dataSettt.pop(4)
    dataSet = dataSett.values.tolist()
    return dataSet, dataSettt

In [46]:
a,b=createDataSet()
attrValues = {'sepal length': ['continuous'], 'sepal width': ['continuous'], 'petal length': ['continuous'], 'petal width': ['continuous']}
print(a)

[[5.1, 3.5, 1.4, 0.2, 'setosa'], [4.9, 3.0, 1.4, 0.2, 'setosa'], [4.7, 3.2, 1.3, 0.2, 'setosa'], [4.6, 3.1, 1.5, 0.2, 'setosa'], [5.0, 3.6, 1.4, 0.2, 'setosa'], [5.4, 3.9, 1.7, 0.4, 'setosa'], [4.6, 3.4, 1.4, 0.3, 'setosa'], [5.0, 3.4, 1.5, 0.2, 'setosa'], [4.4, 2.9, 1.4, 0.2, 'setosa'], [4.9, 3.1, 1.5, 0.1, 'setosa'], [5.4, 3.7, 1.5, 0.2, 'setosa'], [4.8, 3.4, 1.6, 0.2, 'setosa'], [4.8, 3.0, 1.4, 0.1, 'setosa'], [4.3, 3.0, 1.1, 0.1, 'setosa'], [5.8, 4.0, 1.2, 0.2, 'setosa'], [5.7, 4.4, 1.5, 0.4, 'setosa'], [5.4, 3.9, 1.3, 0.4, 'setosa'], [5.1, 3.5, 1.4, 0.3, 'setosa'], [5.7, 3.8, 1.7, 0.3, 'setosa'], [5.1, 3.8, 1.5, 0.3, 'setosa'], [5.4, 3.4, 1.7, 0.2, 'setosa'], [5.1, 3.7, 1.5, 0.4, 'setosa'], [4.6, 3.6, 1.0, 0.2, 'setosa'], [5.1, 3.3, 1.7, 0.5, 'setosa'], [4.8, 3.4, 1.9, 0.2, 'setosa'], [5.0, 3.0, 1.6, 0.2, 'setosa'], [5.0, 3.4, 1.6, 0.4, 'setosa'], [5.2, 3.5, 1.5, 0.2, 'setosa'], [5.2, 3.4, 1.4, 0.2, 'setosa'], [4.7, 3.2, 1.6, 0.2, 'setosa'], [4.8, 3.1, 1.6, 0.2, 'setosa'], [5.4, 3

In [47]:
def splitNumericalDataSet(dataSet, splitPoint=0.0):
    lessThanSet = []
    greaterThanSet = []
    for featVec in dataSet:
        if featVec[-1] <= splitPoint:
            lessThanSet.append(featVec)
        else:
            greaterThanSet.append(featVec)
    return lessThanSet, greaterThanSet

In [48]:
def calcShannonEnt(dataSet, isNumerical=False, splitPoint=None):
    numEntries = len(dataSet)
    labelCounts = set() if isNumerical else {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if isNumerical:
            labelCounts.add(currentLabel)
        else:
            if currentLabel not in labelCounts.keys():
                labelCounts[currentLabel] = 0
            labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    if isNumerical:
        # Handle numerical attributes
        lessThanSet, greaterThanSet = splitNumericalDataSet(dataSet, splitPoint)
        probLess = len(lessThanSet) / numEntries if numEntries != 0 else 0
        probGreater = len(greaterThanSet) / numEntries if numEntries != 0 else 0
        shannonEnt = probLess * calcShannonEnt(lessThanSet, isNumerical=False) + \
                      probGreater * calcShannonEnt(greaterThanSet, isNumerical=False)
    else:
        for key in labelCounts:
            prob = float(labelCounts[key]) / numEntries
            shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [49]:
def gain(unionSet, subsets):
    #input : data and disjoint subsets of it
    #output : information gain
    S = len(unionSet)
    #calculate impurity before split
    impurityBeforeSplit = calcShannonEnt(unionSet)
    #calculate impurity after split
    weights = [len(subset)/S for subset in subsets]
    impurityAfterSplit = 0
    for i in range(len(subsets)):
        impurityAfterSplit += weights[i]*calcShannonEnt(subsets[i])
    #calculate total gain
    totalGain = impurityBeforeSplit - impurityAfterSplit
    return totalGain

In [50]:
print(calcShannonEnt(a))

1.584962500721156


In [51]:
def splitDataSet(dataSet, axis, value, isNumerical=False, splitPoint=0.0):
    retDataSet = []
    for featVec in dataSet:
        if isNumerical:
            if value == "<=" and featVec[axis] <= splitPoint:
                retDataSet.append(featVec)
            elif value == ">" and featVec[axis] > splitPoint:
                retDataSet.append(featVec)
        elif featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis + 1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [52]:
def isAttrDiscrete(attribute):
    if attribute not in b:
        raise ValueError("Attribute not listed")
    elif len(attrValues[attribute]) == 1 and attrValues[attribute][0] == "continuous":
        return False
    else:
        return True

def splitAttribute(curData, curAttributes):
    splitted = []
    maxEnt = -1*float("inf")
    best_attribute = -1
    #None for discrete attributes, threshold value for continuous attributes
    best_threshold = None
    for attribute in curAttributes:
        indexOfAttribute = b.index(attribute)
        if isAttrDiscrete(attribute):
            #split curData into n-subsets, where n is the number of
            #different values of attribute i. Choose the attribute with
            #the max gain
            valuesForAttribute = attrValues[attribute]
            subsets = [[] for a in valuesForAttribute]
            for row in curData:
                for index in range(len(valuesForAttribute)):
                    if row[i] == valuesForAttribute[index]:
                        subsets[index].append(row)
                        break
            e = gain(curData, subsets)
            if e > maxEnt:
                maxEnt = e
                splitted = subsets
                best_attribute = attribute
                best_threshold = None
        else:
            #sort the data according to the column.Then try all
            #possible adjacent pairs. Choose the one that
            #yields maximum gain
            curData.sort(key = lambda x: x[indexOfAttribute])
            for j in range(0, len(curData) - 1):
                if curData[j][indexOfAttribute] != curData[j+1][indexOfAttribute]:
                    threshold = (curData[j][indexOfAttribute] + curData[j+1][indexOfAttribute]) / 2
                    less = []
                    greater = []
                    for row in curData:
                        if(row[indexOfAttribute] > threshold):
                            greater.append(row)
                        else:
                            less.append(row)
                    e = gain(curData, [less, greater])
                    if e >= maxEnt:
                        splitted = [less, greater]
                        maxEnt = e
                        best_attribute = attribute
                        best_threshold = threshold
    return (best_attribute,best_threshold,splitted)

In [53]:
numFeatures = len(a[0]) - 1
baseEntropy = calcShannonEnt(a, isNumerical=False)
bestInfoGain = 0.0
bestFeature = -1
bestSplitPoint = None

for i in range(numFeatures):
    featList = [example[i] for example in a]
    uniqueVals = set(featList)
    newEntropy = 0.0
    if isinstance(featList[0], (int, float)):
        # If numerical attribute, find the best split point
        for value in uniqueVals:
            print(a)
            print(i)
            print(value)
            subDataSet = splitDataSet(a, i, value, isNumerical=True)
            print(subDataSet)

[[5.1, 3.5, 1.4, 0.2, 'setosa'], [4.9, 3.0, 1.4, 0.2, 'setosa'], [4.7, 3.2, 1.3, 0.2, 'setosa'], [4.6, 3.1, 1.5, 0.2, 'setosa'], [5.0, 3.6, 1.4, 0.2, 'setosa'], [5.4, 3.9, 1.7, 0.4, 'setosa'], [4.6, 3.4, 1.4, 0.3, 'setosa'], [5.0, 3.4, 1.5, 0.2, 'setosa'], [4.4, 2.9, 1.4, 0.2, 'setosa'], [4.9, 3.1, 1.5, 0.1, 'setosa'], [5.4, 3.7, 1.5, 0.2, 'setosa'], [4.8, 3.4, 1.6, 0.2, 'setosa'], [4.8, 3.0, 1.4, 0.1, 'setosa'], [4.3, 3.0, 1.1, 0.1, 'setosa'], [5.8, 4.0, 1.2, 0.2, 'setosa'], [5.7, 4.4, 1.5, 0.4, 'setosa'], [5.4, 3.9, 1.3, 0.4, 'setosa'], [5.1, 3.5, 1.4, 0.3, 'setosa'], [5.7, 3.8, 1.7, 0.3, 'setosa'], [5.1, 3.8, 1.5, 0.3, 'setosa'], [5.4, 3.4, 1.7, 0.2, 'setosa'], [5.1, 3.7, 1.5, 0.4, 'setosa'], [4.6, 3.6, 1.0, 0.2, 'setosa'], [5.1, 3.3, 1.7, 0.5, 'setosa'], [4.8, 3.4, 1.9, 0.2, 'setosa'], [5.0, 3.0, 1.6, 0.2, 'setosa'], [5.0, 3.4, 1.6, 0.4, 'setosa'], [5.2, 3.5, 1.5, 0.2, 'setosa'], [5.2, 3.4, 1.4, 0.2, 'setosa'], [4.7, 3.2, 1.6, 0.2, 'setosa'], [4.8, 3.1, 1.6, 0.2, 'setosa'], [5.4, 3

In [54]:
def chooseBestFeatureToSplit(dataSet, labels):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet, isNumerical=False)
    bestInfoGain = 0.0
    bestFeature = -1
    bestSplitPoint = None
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        if isinstance(featList[0], (int, float)):
            # If numerical attribute, find the best split point
            for value in uniqueVals:
                subDataSet = splitDataSet(dataSet, i, value, isNumerical=True)
                prob = len(subDataSet) / float(len(dataSet))
                newEntropy += prob * calcShannonEnt(subDataSet, isNumerical=True)
            infoGain = baseEntropy - newEntropy
        else:
            # If categorical attribute
            for value in uniqueVals:
                subDataSet = splitDataSet(dataSet, i, value)
                prob = len(subDataSet) / float(len(dataSet))
                newEntropy += prob * calcShannonEnt(subDataSet, isNumerical=False)
            infoGain = baseEntropy - newEntropy

        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
            if isinstance(featList[0], (int, float)):
                # If numerical attribute, store the best split point
                bestSplitPoint = value

    return bestFeature, bestSplitPoint


In [55]:
ra = chooseBestFeatureToSplit(a, b)
print(ra)

(0, 7.7)


In [56]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=lambda item: item[1], reverse=True)
    return sortedClassCount[0][0]

In [57]:
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat, bestSplitPoint = chooseBestFeatureToSplit(dataSet, labels)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel: {}}
    del labels[bestFeat]
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    if isinstance(list(uniqueVals)[0], (int, float)):  # Convert set to list here
        for value in ["<=", ">"]:
            subLabels = labels.copy()
            subDataSet = splitDataSet(dataSet, bestFeat, value, isNumerical=True, splitPoint=bestSplitPoint)
            myTree[bestFeatLabel][f"{value} {bestSplitPoint}"] = createTree(subDataSet, subLabels)
    else:
        for value in uniqueVals:
            subLabels = labels.copy()
            myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

In [58]:
dataSet, labels=createDataSet()
tr=createTree(dataSet, labels)
print(tr)

IndexError: list index out of range